# MNIST classification with CNN  

#### from tensorflow tutorials "Deep MNIST for Experts" (https://www.tensorflow.org/versions/r0.10/tutorials/mnist/pros/#a1)

In [17]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import tensorflow as tf 
import numpy as np 
sess = tf.InteractiveSession()

In [1]:
x = tf.placeholder(tf.float32, shape=[None, 784])
x_image = tf.reshape(x, [-1,28,28,1])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

NameError: name 'tf' is not defined

In [4]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [5]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [6]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

In [8]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [9]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [10]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [11]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [20]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
y_conv

<tf.Tensor 'Softmax_1:0' shape=(?, 10) dtype=float32>

In [13]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [14]:
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

### experimenting with gradients manipulation within layers

In [87]:
b = mnist.train.next_batch(50)
print(b[0].shape)
print(b[1].shape)
print(b[0][:1, :].shape)

(50, 784)
(50, 10)
(1, 784)


In [113]:
# y_conv has shape (?, 10), and x_image has shape (?, 28, 28, 1)
wrt = x_image
target = y_conv[-1, 2]
print(target)
# grads = tf.gradients(y_conv, wrt)
grads = tf.gradients(target, wrt)
print(grads)

Tensor("strided_slice_22:0", shape=(), dtype=float32)
[<tf.Tensor 'gradients_22/Conv2D_grad/Conv2DBackpropInput:0' shape=(?, 28, 28, 1) dtype=float32>]


In [114]:
gradients = sess.run([grads], feed_dict={x: b[0][:2, :], y_:b[1][:2, :], keep_prob: 1.0})
print(len(gradients))
print(len(gradients[0]))
gradients[0][0].shape

1
1


(2, 28, 28, 1)

### end of experiment

In [84]:
sess.run(tf.initialize_all_variables())

for i in range(20000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

Instructions for updating:
Use `tf.global_variables_initializer` instead.
step 0, training accuracy 0.02


KeyboardInterrupt: 

In [ ]:
print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))